In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
#Step 1: Load libraries for the U-net Model


import cv2
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
import tensorflow as tf

tf.keras.layers.Conv1D(
    filters,
    kernel_size,
    strides=1,
    padding='valid',
    data_format='channels_last',
    dilation_rate=1,
    groups=1,
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)

In [3]:
def conv_block(x, filter_size, size, stride, batch_norm=False):
    
    conv = layers.Conv1D(size, filter_size, strides = stride, padding="same", kernel_initializer = 'he_normal')(x)
    if batch_norm is True:
        conv = layers.BatchNormalization()(conv)
    conv = layers.Activation("relu")(conv)
    
    return conv

In [197]:
def oleg_model(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):
    inputs1 = layers.Input(input_shape, dtype=tf.float32)
    
    block_1 = conv_block(inputs1, 1, 64, 1, batch_norm=True)
    block_2 = conv_block(block_1, 3, 64, 1, batch_norm=True)
    block_3 = conv_block(block_2, 1, 128, 1, batch_norm=True)
    block_4 = conv_block(block_3, 3, 128, 1, batch_norm=True)
    block_5 = conv_block(block_4, 1, 256, 1, batch_norm=True)
    block_6 = conv_block(block_5, 3, 256, 1, batch_norm=True)
    block_7 = conv_block(block_6, 1, 512, 1, batch_norm=True)
    block_8 = conv_block(block_7, 3, 512, 1, batch_norm=True)
    
    average_pooling = AveragePooling1D(pool_size=2, strides=2, padding='same')(block_8)
    flatten = tf.reshape(average_pooling,[-1])
    oleg_final = layers.Dense(1, activation = "linear")(average_pooling)
    
   
#     conv_final = layers.Conv1D(NUM_CLASSES, kernel_size=(conv)
#     conv_final = layers.BatchNormalization(axis=3)(conv_final)
#     conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel
    
    model = models.Model(inputs1, oleg_final, name="TargetModel")
    return model

In [198]:
model = oleg_model(input_shape = (2,1500))

In [199]:
model.summary()

Model: "TargetModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 2, 1500)]         0         
_________________________________________________________________
conv1d_57 (Conv1D)           (None, 2, 64)             96064     
_________________________________________________________________
batch_normalization_56 (Batc (None, 2, 64)             256       
_________________________________________________________________
activation_62 (Activation)   (None, 2, 64)             0         
_________________________________________________________________
conv1d_58 (Conv1D)           (None, 2, 64)             12352     
_________________________________________________________________
batch_normalization_57 (Batc (None, 2, 64)             256       
_________________________________________________________________
activation_63 (Activation)   (None, 2, 64)             

In [7]:
import pandas as pd

In [15]:
dataset = pd.read_csv(r"dataset.csv")

In [17]:
dataset = dataset.iloc[:,1:]
dataset

,sample,degree_x,degree_y,is_valid,y
0,0,1.025671,-3.618744,0,0
1,1,1.025671,-3.615809,0,0
2,2,1.034486,-3.615809,0,0
3,3,1.049177,-3.609939,0,0
4,4,1.060930,-3.601134,0,0
...,...,...,...,...,...
17999995,14995,-5.488272,-1.162959,0,2
17999996,14996,-5.485360,-1.165905,0,2
17999997,14997,-5.482448,-1.168850,0,2
17999998,14998,-5.479535,-1.165905,0,2


In [97]:
x[2:5]

array([[ 1.034486, -3.615809],
       [ 1.049177, -3.609939],
       [ 1.06093 , -3.601134]])

In [182]:
prev_i = 0
i = 0
first = 1
count_1 = 0
count_2 = 0
c1 = []
c2= []
x = []
y = []
x_train = []
y_train = []
while prev_i < 18000000:
    while i+1500 < 15000:        
        window_x = dataset.iloc[i:i+1500, 1].to_list()
        window_y = dataset.iloc[i:i+1500, 2].to_list()
        x.append([window_x, window_y])
        y.append(dataset.iloc[i, -1])
        first = 0
        i = i + 125
        c1 = c1+ [len(x)]     
    
    x_train = x_train + x
    y_train = y_train + y
    c2 = c2+ [len(x_train[len(x_train)-1])]
#     c1 = c1 + [count_1]    
    prev_i = prev_i + i
    


In [183]:
X_train1 = np.array(x_train)
y_train1 = np.array(y_train)

In [184]:
X_train1.shape

(144072, 2, 1500)

In [185]:
y_train1.shape

(144072,)

In [93]:
from tensorflow.keras.losses import MSE

In [200]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [201]:
model.fit(X_train1, y_train1, epochs=10, batch_size = 16)


Epoch 1/10
2030/9005 [=====>........................] - ETA: 40s - loss: 0.0204

KeyboardInterrupt: 

In [180]:
16*1500

24000

In [164]:
model.summary()

Model: "TargetModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 2, 1500)]         0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 2, 64)             96064     
_________________________________________________________________
batch_normalization_16 (Batc (None, 2, 64)             256       
_________________________________________________________________
activation_18 (Activation)   (None, 2, 64)             0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 2, 64)             12352     
_________________________________________________________________
batch_normalization_17 (Batc (None, 2, 64)             256       
_________________________________________________________________
activation_19 (Activation)   (None, 2, 64)             